In [1]:
import numpy as np

from scipy.linalg import solve_continuous_are

# from darli.models import RobotModel
import casadi as cs

In [2]:
H = np.vstack((np.zeros((1, 3)), np.eye(3)))

def hat(v):
    """
    Generate the skew-symmetric matrix (hat operator) for a 3D vector.
    
    Args:
        v (cs.SX): A 3-element CasADi symbolic column vector.

    Returns:
        cs.SX: A 3x3 skew-symmetric matrix.
    """
    return cs.blockcat([[0, -v[2], v[1]],
            [v[2], 0, -v[0]],
            [-v[1], v[0], 0]])


def L(q):
    """
    Generate the left quaternion multiplication matrix.
    
    Args:
        q (cs.SX): A 4-element quaternion represented as a CasADi symbolic column vector.

    Returns:
        cs.SX: A 4x4 matrix for left quaternion multiplication.
    """
    s = q[0]
    v = q[1:4]
    return cs.vertcat(
        cs.horzcat(s, -v.T),
        cs.horzcat(v, s * cs.SX.eye(3) + hat(v))
    )

def G(q):
    """
    Generate the matrix for quaternion and angular velocity multiplication.
    
    Args:
        q (cs.SX): A 4-element quaternion represented as a CasADi symbolic column vector.

    Returns:
        cs.SX: A 4x3 matrix used in quaternion rate calculations.
    """
    
    return L(q) @ H

def rptoq(phi):
    """
    Convert a rotation vector to a quaternion.
    
    Args:
        phi (cs.SX): A 3-element rotation vector represented as a CasADi symbolic column vector.

    Returns:
        cs.SX: A quaternion represented as a 4-element CasADi symbolic column vector.
    """
    norm_phi = cs.sqrt(cs.dot(phi, phi))
    return cs.vertcat(1, phi) / cs.sqrt(1 + norm_phi**2)

def qtorp(q):
    """
    Convert a quaternion to a rotation vector.
    
    Args:
        q (cs.SX): A quaternion represented as a 4-element CasADi symbolic column vector.

    Returns:
        cs.SX: A 3-element rotation vector represented as a CasADi symbolic column vector.
    """
    return q[1:4] / q[0]

def qtoR(q):
    """
    Generate the rotation matrix from a quaternion.
    
    Args:
        q (cs.SX): A quaternion represented as a 4-element CasADi symbolic column vector.

    Returns:
        cs.SX: A 3x3 rotation matrix.
    """
    T = np.eye(4)
    T[1, 1] = -1
    T[2, 2] = -1
    T[3, 3] = -1
    return H.T @ T @ L(q) @ T @ L(q) @ H

def E(state):
    """
    Construct the state transition matrix for the quadrotor system.
    
    Args:
        q (cs.SX): A quaternion represented as a 4-element CasADi symbolic column vector.

    Returns:
        cs.SX: A 13x13 state transition matrix.
    """
    q = state[3:7]      # Orientation (quaternion)
    return cs.blockcat([[cs.SX.eye(3), cs.SX.zeros(3, 3), cs.SX.zeros(3, 6)], 
             [cs.SX.zeros(4, 3), G(q), cs.SX.zeros(4, 6)],
             [cs.SX.zeros(6, 3), cs.SX.zeros(6, 3), cs.SX.eye(6)]])

    


In [3]:
# Quadrotor parameters
m = 0.5
l = 0.1750
J = np.diag([0.0023, 0.0023, 0.004])
g = 9.81
kt = 1.0
km = 0.0245

# Thrust force mapping
thrust_mapping = np.array([[0, 0, 0, 0],
                           [0, 0, 0, 0],
                           [kt, kt, kt, kt]])

# Torque mapping
torque_mapping = np.array([[0, l*kt, 0, -l*kt],
                           [-l*kt, 0, l*kt, 0],
                           [km, -km, km, -km]])

In [4]:
def quad_dynamics(x, u):
    """
    Compute the dynamics of the quadrotor.
    
    Args:
        x (cs.SX): The state vector of the quadrotor, including position, orientation (quaternion), linear velocity, and angular velocity.
        u (cs.SX): The control input vector for the quadrotor (e.g., motor thrusts).

    Returns:
        cs.SX: The time derivative of the state vector.
    """
    # Extract the components of the state vector
    r = x[0:3]      # Position
    q = x[3:7]      # Orientation (quaternion)
    v = x[7:10]     #Linear velocity
    omega = x[10:13]  # Angular velocity

    # Normalize the quaternion to prevent drift over time
    q = q / cs.norm_2(q)

    # Convert quaternion to rotation matrix for world to body frame transformation
    R = qtoR(q)

    # Compute the derivatives of the state vector
    r_dot = R @ v
    q_dot = 0.5 * L(q) @ H @ omega
    v_dot = R.T @ np.array([0,0,-g]) - hat(omega) @ v + (1 / m) * thrust_mapping @ u 
    omega_dot = cs.solve(J, -hat(omega) @ J @ omega + torque_mapping@u)

    # Concatenate derivatives into a single column vector
    return cs.vertcat(r_dot, q_dot, v_dot, omega_dot)


In [5]:
# Control inputs (thrusts from the 4 motors)
u = cs.SX.sym('u', 4)
# State variables
x = cs.SX.sym('x', 13) # Assuming the state vector x contains position (3), orientation (quaternion, 4), linear velocity (3), and angular velocity (3)
h = cs.SX.sym('h', 1)
q = x[3:7]


In [6]:
# RK4 integration for quadrotor dynamics
def floating_dynamics_rk4(x, u, h):
    # Define the Runge-Kutta 4th order stages
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5 * h * f1, u)
    f3 = quad_dynamics(x + 0.5 * h * f2, u)
    f4 = quad_dynamics(x + h * f3, u)

    # Combine the stages to compute the final state after one time step
    xn = x + (h / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)

    # Re-normalize the quaternion part of the state vector
    # CasADi has a different syntax for vector/matrix norms: use ca.norm_2
    q_norm = cs.norm_2(xn[3:7])
    xn[3:7] = xn[3:7] / q_norm

    return xn

In [7]:
# Create a CasADi function for the RK4 integration step
quad_dynamics_rk4_step = cs.Function('quad_dynamics_rk4_step', [x, u, h], [floating_dynamics_rk4(x, u, h)])

In [8]:

# Calculate the Jacobians using CasADi
A_casadi = cs.jacobian(quad_dynamics_rk4_step(x, u, h), x)
B_casadi = cs.jacobian(quad_dynamics_rk4_step(x, u, h), u)

# Create functions to calculate the Jacobians
A_func = cs.Function('A_func', [x, u, h], [A_casadi])
B_func = cs.Function('B_func', [x, u, h], [B_casadi])
E_func = cs.Function('E_func', [x], [E(x)])
L_func = cs.Function('L_func', [q], [L(q)])

In [9]:
# Simulation parameters
dt = 0.001
Nx = 13
Nxt = 12
Nu = 4
Tfinal = 5.0
Nt = int(Tfinal / dt) + 1
thist = np.arange(0, dt * (Nt - 1), step=dt)

In [10]:
# Initial Conditions
u0 = (m * g / 4) * np.ones(4)
r0 = np.array([0.0, 0, 1.0])
q0 = np.array([1.0, 0, 0, 0])
v0 = np.zeros(3)
omega0 = np.zeros(3)
x0 = np.concatenate((r0, q0, v0, omega0))

In [11]:
A = np.array(A_func(x0, u0, dt))
B = np.array(B_func(x0, u0, dt))

In [12]:
# Reduced system (assuming the E(q) function returns a numpy array)
A_r = np.array(E_func(x0).T @ A_func(x0, u0, dt) @ E_func(x0))
B_r = np.array(E_func(x0).T @ B_func(x0, u0, dt))

In [13]:
from scipy.linalg import solve_discrete_are as dare

def dlqr(A, B, Q, R):
    '''Discrete time LTI LQR'''
    # Solve discrete Ricatti equation (DARE)
    P = dare(A, B, Q, R)
    # Compute the LQR gain
    K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)
    return K, P


In [14]:

# Cost weights
Q = np.eye(Nxt)
R = 0.1 * np.eye(Nu)

# LQR Controller
K, P = dlqr(A_r, B_r, Q, R)

# Feedback controller
def controller(x):
    q0 = x0[3:7]
    q = x[3:7]
    q_error = L_func(q0).T @ q
    phi = np.array(qtorp(q_error)).reshape(3,)
    
    delta_x_tilde = np.concatenate((x[0:3] - r0, phi, x[7:10] - v0, x[10:13] - omega0))
    
    u = u0 - K @ delta_x_tilde
    return u



In [15]:
# Simulation
uhist = np.zeros((Nu, Nt))
xhist = np.zeros((Nx, Nt))
xhist[:, 0] = np.concatenate((r0 + np.random.randn(3), np.array(L_func(q0) @ rptoq(np.array([1, 0, 0]))).reshape(4,), v0, omega0))
for k in range(Nt - 1):
    uhist[:, k] = controller(xhist[:, k])
    xhist[:, k + 1] = np.array(quad_dynamics_rk4_step(xhist[:, k], uhist[:, k], dt)).reshape(13,)

In [16]:
xhist[:,-1]

array([-2.38311627e-03, -1.67415821e-02,  9.94644818e-01,  9.99998783e-01,
        8.82828417e-04, -1.08109258e-04,  1.28150079e-03,  2.30156980e-03,
        1.67969209e-02,  5.37955067e-03, -1.90038227e-03,  2.11552862e-04,
       -1.28160942e-03])